In [24]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import glm

from sklearn.metrics import mean_absolute_error,mean_squared_error, accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#### correlated feature removed 

In [3]:
df=pd.read_csv(r'C:\Melbourne_Escooter\RealDataset\FinalResults\SA1\Features-trips\Final\AllFeatures-original - wholeNumberDensity -removed0.7.csv')

In [4]:
column=['tripDensity']
df_depe=df.drop(column, axis=1)
df_depe

,SA1_CODE21,date,hour,trainDensity,busDensity,tramDensity,Cafe perc,Office perc,Shops perc,mxi,recreationCount,campusCount,5-14 %,40-49 %,50-64 %,male%,Population density 2021,rainfall,min_temp
0,20604111702,1,0,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,7.422579,8.152377,49.511571,9456.2,0,7.4
1,20604111702,1,1,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,7.422579,8.152377,49.511571,9456.2,0,7.4
2,20604111702,1,2,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,7.422579,8.152377,49.511571,9456.2,0,7.4
3,20604111702,1,3,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,7.422579,8.152377,49.511571,9456.2,0,7.4
4,20604111702,1,4,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,7.422579,8.152377,49.511571,9456.2,0,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56611,20604151016,7,19,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,13.562667,0.000000,51.647198,9238.5,1,8.0
56612,20604151016,7,20,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,13.562667,0.000000,51.647198,9238.5,1,8.0
56613,20604151016,7,21,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,13.562667,0.000000,51.647198,9238.5,1,8.0
56614,20604151016,7,22,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,13.562667,0.000000,51.647198,9238.5,1,8.0


In [5]:
df_depe=df_depe.drop(['SA1_CODE21'], axis=1)

In [6]:
df_depe["hour"].replace({0:1,1:1,2:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4,11:4,12:5,13:5,14:5,15:6,16:6,17:6,18:7,19:7,20:7,21:8,22:8,23:8}, inplace=True)
df_depe["date"].replace({1:1,2: 1, 3: 1,4: 1,5: 1,6: 2,7: 2}, inplace=True)
df_depe['trainDensity'] = df['trainDensity'].map(lambda x: 1 if x>0 else 0)

In [7]:
categorical =['hour','date','rainfall','trainDensity']
for col in categorical:
    df_depe[col]=df_depe[col].astype('category')

In [8]:
df_depe=pd.get_dummies(df_depe, drop_first=True)
df_depe.columns

Index(['busDensity', 'tramDensity', 'Cafe perc', 'Office perc', 'Shops perc',
       'mxi', 'recreationCount', 'campusCount', '5-14 %', '40-49 %', '50-64 %',
       'male%', 'Population density 2021', 'min_temp', 'date_2', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8',
       'trainDensity_1', 'rainfall_1'],
      dtype='object')

In [9]:
scale= StandardScaler()
df_sc= scale.fit_transform(df_depe)
df_sc=pd.DataFrame(df_sc, columns=df_depe.columns)

In [10]:
df['tripDensity'] = df['tripDensity'].map(lambda x: 1 if x>49.0 else 0)
df['tripDensity']=df['tripDensity'].astype('category')

In [11]:
y=df['tripDensity']
X=df_sc

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [31]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1,random_state=42,max_iter=1000)

In [32]:
model = clf.fit(X_train, y_train)

In [15]:
ypredict = clf.predict(X_test)

In [16]:
print("MAE %f" %mean_absolute_error(y_test, ypredict))


MAE 0.197015


In [17]:
print("MSE  %f" %mean_squared_error(y_test, ypredict))

MSE  0.197015


In [19]:
import numpy as np
print("RMSE  %f" %np.sqrt(mean_squared_error(y_test, ypredict)))

RMSE  0.443864


In [21]:
f1_score(y_test, ypredict)

0.231484671029969

In [25]:
y_pred_proba = clf.predict_proba(X_test)[::,1]
auc =roc_auc_score(y_test, y_pred_proba)
print(auc)

0.7807389646753395


In [26]:
import statsmodels.api as sm
X_train =sm.add_constant(X_train)

In [27]:
model=sm.Logit(y_train,X_train)

In [28]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.411080
         Iterations 7


In [29]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            tripDensity   No. Observations:                45292
Model:                          Logit   Df Residuals:                    45267
Method:                           MLE   Df Model:                           24
Date:                Thu, 22 Sep 2022   Pseudo R-squ.:                  0.1749
Time:                        11:23:12   Log-Likelihood:                -18619.
converged:                       True   LL-Null:                       -22565.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -1.8517      0.017   -107.478      0.000      -1.885      -1.818
busDensity                 -0.1110      0.014     -7.880      0.000      -0.139      -0.083
tramDensity                 0.1892      0.015     12.928      0.000       0.160       0.218
Cafe perc                   0.3764      0.016     23.367      0.000       0.345       0.408
Office perc                 0.1284      0.011     11.638      0.000       0.107       0.150
Shops perc                  0.1312      0.012     11.319      0.000       0.108       0.154
mxi                         0.2070      0.019     10.687      0.000       0.169       0.245
recreationCount            -0.2205      0.029     -7.704      0.000      -0.277      -0.164
campusCount                -0.0380      0.013     -2.863      0.004      -0.064      -0.012
5-14 %                     -0.1989      0.024     -8.245      0.000      -0.246      -0.152
40-49 %                    -0.3537      0.024    -14.524      0.000      -0.401      -0.306
50-64 %                     0.1005      0.022      4.480      0.000       0.057       0.144
male%                       0.5454      0.019     28.029      0.000       0.507       0.584
Population density 2021     0.1999      0.020     10.032      0.000       0.161       0.239
min_temp                    0.1790      0.018      9.880      0.000       0.143       0.214
date_2                      0.3914      0.020     20.025      0.000       0.353       0.430
hour_2                     -0.3035      0.020    -15.051      0.000      -0.343      -0.264
hour_3                     -0.1857      0.019     -9.895      0.000      -0.223      -0.149
hour_4                     -0.0079      0.017     -0.453      0.650      -0.042       0.026
hour_5                      0.1170      0.017      6.991      0.000       0.084       0.150
hour_6                      0.2180      0.016     13.304      0.000       0.186       0.250
hour_7                      0.1769      0.016     10.736      0.000       0.145       0.209
hour_8                      0.1592      0.017      9.610      0.000       0.127       0.192
trainDensity_1             -0.0614      0.015     -4.125      0.000      -0.091      -0.032
rainfall_1                 -0.1428      0.015     -9.254      0.000      -0.173      -0.113
===========================================================================================
"""

In [29]:
# cross validation

In [33]:
test_score= cross_val_score(model, X_test, y_test, scoring='neg_mean_squared_error') #run sklearn model again to override statmodel model

In [34]:
np.mean(np.absolute(test_score))

0.19754471563740744